# Clasificador de corredores de F1
Este trabajo tiene por objetivo que dado un gran dataset poder clasificar a los corredores de la formula 1, la idea es ocupar todo el conocimiento obtenido dentro del curso para lograr llegar a este objetivo.
Para esto, la plataforma tiene algunas preguntas a responder, las cuales son:
- ¿Qué tipo de preguntas sobre ciencia de datos puedes responder en función de tus datos?
puedo generar algunos algoritmos de clasificación y compararlos entre ellos, junto con que tipo de algoritmo entrega un mejor recall al ser optimizado
- ¿Es esta una tarea supervisada o no supervisada?
Como tal, no tiene una columna target, pero yo voy a hacer la columna target definiendo los corredores
- Si se trata de una tarea supervisada, ¿es una tarea de regresión o clasificación?
De clasificación
- ¿Qué tan limpios están tus datos?
yo espero q lo más limpio posibles
- ¿Qué problemas crees que pueden surgir?
quizas en temas de tiempo se demore bastante, debido a la dimension del dataset

# exploración de datos

## importando las librerias a usar

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.model_selection import GridSearchCV

## cargando la data

Las ubicaciones

In [3]:
archivo1='/Users/matiasinostrozacifuentes/Desktop/python_intro/F1_ML_classifier/Data/results.csv'
archivo2='/Users/matiasinostrozacifuentes/Desktop/python_intro/F1_ML_classifier/Data/status.csv'
archivo3='/Users/matiasinostrozacifuentes/Desktop/python_intro/F1_ML_classifier/Data/drivers.csv'
archivo4='/Users/matiasinostrozacifuentes/Desktop/python_intro/F1_ML_classifier/Data/constructors.csv'
archivo5='/Users/matiasinostrozacifuentes/Desktop/python_intro/F1_ML_classifier/Data/driver_standings.csv'
archivo6='/Users/matiasinostrozacifuentes/Desktop/python_intro/F1_ML_classifier/Data/races.csv'

Cargandolas

In [4]:
resultados=pd.read_csv(archivo1)
Estado=pd.read_csv(archivo2)
drivers=pd.read_csv(archivo3)
cosntructors=pd.read_csv(archivo4)
standing=pd.read_csv(archivo5)
races=pd.read_csv(archivo6)

In [5]:
pd.get_option("display.max_columns",None)

20

primer vistazo

In [6]:
resultados.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [7]:
Estado.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [8]:
drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [9]:
cosntructors.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [10]:
standing.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


In [11]:
races.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,29/03/09,6:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,05/04/09,9:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,19/04/09,7:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,26/04/09,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,10/05/09,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


A primera impresion hay algunas columnas que hay que eliminar como el url dado a que solo dificultarian en el proceso de trabajo

Revisando si hay NaN

In [ ]:
resultados
Estado
drivers
cosntructors
standing
races

In [12]:
resultados.isna().sum()


resultId           0
raceId             0
driverId           0
constructorId      0
number             0
grid               0
position           0
positionText       0
positionOrder      0
points             0
laps               0
time               0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
statusId           0
dtype: int64

In [13]:
Estado.isna().sum()


statusId    0
status      0
dtype: int64

In [14]:
drivers.isna().sum()


driverId       0
driverRef      0
number         0
code           0
forename       0
surname        0
dob            0
nationality    0
url            0
dtype: int64

In [15]:
cosntructors.isna().sum()


constructorId     0
constructorRef    0
name              0
nationality       0
url               0
dtype: int64

In [16]:
standing.isna().sum()


driverStandingsId    0
raceId               0
driverId             0
points               0
position             0
positionText         0
wins                 0
dtype: int64

In [17]:
races.isna().sum()

raceId       0
year         0
round        0
circuitId    0
name         0
date         0
time         0
url          0
dtype: int64

no hay NAN, se eliminaran duplicados

In [18]:
resultados.drop_duplicates(inplace=True)


In [19]:
Estado.drop_duplicates(inplace=True)


In [20]:
drivers.drop_duplicates(inplace=True)


In [21]:
cosntructors.drop_duplicates(inplace=True)


In [22]:
standing.drop_duplicates(inplace=True)


In [23]:
races.drop_duplicates(inplace=True)